In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os


# Define paths
dataset_path = "/teamspace/studios/this_studio/images/PERSONALITIES"

# Data augmentation and loading
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'

)



2024-10-04 03:29:25.237705: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-04 03:29:25.402834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 03:29:25.476596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 03:29:25.492863: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 03:29:25.630240: I tensorflow/core/platform/cpu_feature_guar

Found 2327 images belonging to 88 classes.
Found 537 images belonging to 88 classes.


In [2]:
pip install tensorflow keras_facenet

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_facenet import FaceNet

# Load pre-trained FaceNet model
facenet = FaceNet()
base_model = facenet.model

# Add custom layers
x = base_model.output
# x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Define model
model_facenet = Model(inputs=base_model.input, outputs=predictions)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model_facenet.compile(optimizer=Adam(learning_rate=0.001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_facenet_model.keras', save_best_only=True, monitor='val_accuracy')

# Train the model
history = model_facenet.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[early_stopping, model_checkpoint]
)

# Fine-tuning (optional)
# Unfreeze some layers for fine-tuning
for layer in model_facenet.layers[-20:]:
    layer.trainable = True

# Recompile the model
model_facenet.compile(optimizer=Adam(learning_rate=0.0001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

# Train the model again (fine-tuning)
history_fine_tuning = model_facenet.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stopping, model_checkpoint]
)

I0000 00:00:1728012580.911062    1925 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728012581.077648    1925 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728012581.077891    1925 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728012581.080239    1925 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/100


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1728012595.517578    3289 service.cc:146] XLA service 0x7f865c002390 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728012595.517840    3289 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-10-04 03:29:55.844799: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-04 03:29:58.004070: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


 2/73 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - accuracy: 0.0312 - loss: 4.4693  

I0000 00:00:1728012605.009969    3289 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


73/73 ━━━━━━━━━━━━━━━━━━━━ 65s 629ms/step - accuracy: 0.0997 - loss: 4.3398 - val_accuracy: 0.2998 - val_loss: 3.6313
Epoch 2/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 29s 371ms/step - accuracy: 0.3131 - loss: 3.3646 - val_accuracy: 0.4004 - val_loss: 2.8395
Epoch 3/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 29s 364ms/step - accuracy: 0.3983 - loss: 2.7464 - val_accuracy: 0.4804 - val_loss: 2.4622
Epoch 4/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 29s 362ms/step - accuracy: 0.5059 - loss: 2.2523 - val_accuracy: 0.5102 - val_loss: 2.2273
Epoch 5/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 28s 360ms/step - accuracy: 0.5430 - loss: 2.0662 - val_accuracy: 0.5363 - val_loss: 2.1298
Epoch 6/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 28s 360ms/step - accuracy: 0.5702 - loss: 1.9253 - val_accuracy: 0.5400 - val_loss: 2.0605
Epoch 7/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 27s 341ms/step - accuracy: 0.5866 - loss: 1.8279 - val_accuracy: 0.5326 - val_loss: 2.0488
Epoch 8/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 27s 347ms/step - accuracy: 0.6115 - loss: 1.6992 - val_accuracy

In [4]:
model_facenet.save('facenet_87img_profile_face1.h5')